In [ ]:
# Import libaries 
import numpy as np
import pandas as pd
import sys
import json
import time
import requests
import string
import regex as re
from flatten_json import flatten
from time import sleep

# Like most API connections you will need to goto https://www.setlist.fm/signin create an account and request access to the API
# This shouldn't take too long to retrieve

# Input your setlist api key below
setlist_api = 'xxxx'

# I will use the biffy
biffy_mbid = '892500b7-09a6-4049-ba92-2d192dd70563' # taken from viewing source page on setlist website


# Get the most recent page setlist from website
def get_setlists(artist, api_key):
    r = requests.get('https://api.setlist.fm/rest/1.0/search/setlists?artistName=' + artist, 
                     headers={'Accept': 'application/json', 'x-api-key': api_key})
    return r.json()


# The below website was helpful when flattening the json file
# https://stackoverflow.com/questions/52795561/flattening-nested-json-in-pandas-data-frame

# Use the below code to retrieve all the setlists from the # of pages that you want. 
# You will need to go to that artist's website to see how many pages they have

def get_all_setlists(artist_mbid, api_key, pages):
    result = []
    for page in range(1, pages):
        
        # pull in api data from page
        r = requests.get('https://api.setlist.fm/rest/1.0/artist/' + artist_mbid + '/setlists?p=' + str(page), 
                                headers={'Accept': 'application/json', 'x-api-key': api_key})
        
        # convert api data to json
        rjs = r.json()
        
        # flatten json file to pandas dataframe
        try:
            df = pd.DataFrame( [flatten(x) for x in rjs['setlist'] ])
            df['page'] = page
        except:
            pass
        
        # stack dataframe to dataframe
        result.append(df)
        
        # use the sleep function so calls between API pages are delayed
        sleep(3)
        print('page: ',page, ' completed')
        
    #combine dataframes
    setlists = pd.concat(result)
    
    #convert eventDate to date variable
    setlists['eventDate'] = pd.to_datetime(setlists['eventDate'], format='%d-%m-%Y')
    
    return setlists

# Run Function
final = get_all_setlists(biffy_mbid, setlist_api, 164)
display(final)

# Save Dataframe to CSV
final.to_csv(r'file_location/artist_setlist_data.csv', index = False, header=True)
